## Notebook para la exploración de los ficheros de datos del TFM
### Script para la exploración de los datasets para el TFM: Optimización de la toma de decisiones empresarial
### mediante el análisis de datos

### 1. Carga de los ficheros y exploración de las variables

In [47]:
# Importación de librerías
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
# Lectura de los .CSV y convertimos los tipos automáticamente
df = pd.read_csv('C:/Users/UX530/Desktop/TFM-GIT/Origin_Files/extended_cto_2025-09-30_09_35.csv', delimiter = ';')

In [49]:
df.shape

(11505, 28)

In [50]:
df.dtypes

ID                                 int64
CTO                               object
Código CTO                        object
Estado                            object
Tipo de ampliación                object
Ticket Jira                       object
EC                                object
Fecha de solicitud                object
Fecha de ejecución                object
Fecha de documentación            object
Fecha de parada                   object
Fecha de reanudación              object
Fecha Documentación Rechazada     object
Fecha  Documentación Reparada     object
Fecha de finalización             object
Fecha Cancelación                 object
SLA                               object
Geotipo                           object
Nueva CTO                         object
Rechazos (iteraciones)           float64
Provincia                         object
Población                         object
Zona                             float64
Fase                              object
Cluster         

### Se hará una copia del dataframen original

In [113]:
df_CTO = df.copy()  # Se hará una copia del dataframen original


In [84]:
# Cabecera del dataset
df_CTO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11505 entries, 0 to 11504
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ID                             11505 non-null  int64  
 1   CTO                            11505 non-null  object 
 2   Código CTO                     11505 non-null  object 
 3   Estado                         11505 non-null  object 
 4   Tipo de ampliación             11505 non-null  object 
 5   Ticket Jira                    3459 non-null   object 
 6   EC                             11505 non-null  object 
 7   Fecha de solicitud             11505 non-null  object 
 8   Fecha de ejecución             10208 non-null  object 
 9   Fecha de documentación         10507 non-null  object 
 10  Fecha de parada                2016 non-null   object 
 11  Fecha de reanudación           1829 non-null   object 
 12  Fecha Documentación Rechazada  1526 non-null  

In [85]:
df_CTO.describe()

,ID,Rechazos (iteraciones),Zona
count,11505.000000,1548.000000,11501.000000
mean,21617.206345,0.921835,2.417442
std,10889.299335,0.415864,1.669336
min,1.000000,0.000000,1.000000
25%,13018.000000,1.000000,1.000000
50%,24395.000000,1.000000,1.000000
75%,30983.000000,1.000000,4.000000
max,35816.000000,3.000000,5.000000


### Información dataset CTO ampliaciones

In [86]:
# Número de filas y columnas
print("Número de filas y columnas:", df_CTO.shape)

# Nombre de las columnas
print("Nombres de las columnas:", df_CTO.columns.tolist())

# Tipo de datos de cada columna
print("Tipos de datos:\n", df_CTO.dtypes)

# Información general del DataFrame
print("Información del DataFrame:" , df_CTO.info())

Número de filas y columnas: (11505, 28)
Nombres de las columnas: ['ID', 'CTO', 'Código CTO', 'Estado', 'Tipo de ampliación', 'Ticket Jira', 'EC', 'Fecha de solicitud', 'Fecha de ejecución', 'Fecha de documentación', 'Fecha de parada', 'Fecha de reanudación', 'Fecha Documentación Rechazada', 'Fecha  Documentación Reparada', 'Fecha de finalización', 'Fecha Cancelación', 'SLA', 'Geotipo', 'Nueva CTO', 'Rechazos (iteraciones)', 'Provincia', 'Población', 'Zona', 'Fase', 'Cluster', 'Proveedor OLT', 'Observaciones', 'Activo']
Tipos de datos:
 ID                                 int64
CTO                               object
Código CTO                        object
Estado                            object
Tipo de ampliación                object
Ticket Jira                       object
EC                                object
Fecha de solicitud                object
Fecha de ejecución                object
Fecha de documentación            object
Fecha de parada                   object
Fecha d

#### Columnas de los datasets candidatas para dejar fuera del modelo por tener más de un 60% de nulos

In [114]:
# Calacula el porcentaje de valores nulos en cada columna, y nos quedamos con las que tienen más del 60%
# serán columnas candidatas a eliminar del modelo. 
Columns_delete_CTO = (df_CTO.isnull().mean() * 100)[df_CTO.isnull().mean() * 100 > 60].index.tolist()
Columns_delete_CTO

['Ticket Jira',
 'Fecha de parada',
 'Fecha de reanudación',
 'Fecha Documentación Rechazada',
 'Fecha  Documentación Reparada',
 'Fecha Cancelación',
 'Nueva CTO',
 'Rechazos (iteraciones)',
 'Observaciones']

In [115]:
# Elimanamos las columnas con más del 60% de valores nulos y nos quedamos con el nuevo dataframe
df_CTO_cleaned = df_CTO.drop(columns=Columns_delete_CTO)

### Vamos a transformar las variables fechas que están en tipo object al formato datatime

In [116]:
df_CTO_cleaned['Fecha de ejecución'].isna().sum()

np.int64(1297)

In [117]:
df_CTO_cleaned['Fecha de ejecución']

0        2019-03-28 09:26:11
1                        NaN
2        2019-02-15 12:50:03
3                        NaN
4        2019-02-15 13:09:21
                ...         
11500                    NaN
11501                    NaN
11502                    NaN
11503                    NaN
11504                    NaN
Name: Fecha de ejecución, Length: 11505, dtype: object

In [118]:
df_CTO_cleaned["Fecha de ejecución"] = pd.to_datetime(df_CTO_cleaned["Fecha de ejecución"], format="%Y-%m-%d %H:%M:%S", errors="coerce")

In [119]:
df_CTO_cleaned['Fecha de ejecución']

0       2019-03-28 09:26:11
1                       NaT
2       2019-02-15 12:50:03
3                       NaT
4       2019-02-15 13:09:21
                ...        
11500                   NaT
11501                   NaT
11502                   NaT
11503                   NaT
11504                   NaT
Name: Fecha de ejecución, Length: 11505, dtype: datetime64[ns]

In [120]:
# Columnas con información de fechas relevantes para el análisis
Columns_date = ['Fecha de solicitud','Fecha de documentación', 'Fecha de finalización']

# Transformamos las variables fechas que están en tipo object al formato datatime
for col in Columns_date:
    try:
        df_CTO_cleaned[col] = pd.to_datetime(df_CTO_cleaned[col], errors='coerce', format='%Y-%m-%d')
    except (ValueError, TypeError):
        pass  # Si no se puede convertir, la deja como está



In [121]:
# Verificamos la conversión
df_CTO_cleaned.dtypes

ID                                 int64
CTO                               object
Código CTO                        object
Estado                            object
Tipo de ampliación                object
EC                                object
Fecha de solicitud        datetime64[ns]
Fecha de ejecución        datetime64[ns]
Fecha de documentación    datetime64[ns]
Fecha de finalización     datetime64[ns]
SLA                               object
Geotipo                           object
Provincia                         object
Población                         object
Zona                             float64
Fase                              object
Cluster                           object
Proveedor OLT                     object
Activo                            object
dtype: object

### El resto de nulos de nuestros variables numércias vamos a rellenarlos con la función interpolate()
### Para las variables categícas vamos a utilizar el valor de la moda, el dato que más se repita.

In [122]:
total_rows = len(df_CTO_cleaned)
null_summary = df_CTO_cleaned.isna().sum().to_frame("Nulos")
null_summary["% Nulos"] = (null_summary["Nulos"] / total_rows) * 100
print(null_summary)

                        Nulos    % Nulos
ID                          0   0.000000
CTO                         0   0.000000
Código CTO                  0   0.000000
Estado                      0   0.000000
Tipo de ampliación          0   0.000000
EC                          0   0.000000
Fecha de solicitud          0   0.000000
Fecha de ejecución       1297  11.273359
Fecha de documentación    998   8.674489
Fecha de finalización    1129   9.813125
SLA                         0   0.000000
Geotipo                   945   8.213820
Provincia                   0   0.000000
Población                   0   0.000000
Zona                        4   0.034767
Fase                     1674  14.550196
Cluster                   138   1.199478
Proveedor OLT              11   0.095611
Activo                      0   0.000000


In [ ]:
# Interpolación de los valores nulos en las columnas numéricas
df_CTO_cleaned_interpolated = df_CTO_cleaned.interpolate(method='linear', limit_direction='both')

C:\Users\UX530\AppData\Local\Temp\ipykernel_16340\1582214595.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_CTO_cleaned_interpolated = df_CTO_cleaned.interpolate(method='linear', limit_direction='both')


In [126]:
# Interpolación de los valores nulos en las columnas categóricas con el método de la moda
for col in df_CTO_cleaned_interpolated.select_dtypes(include=['object']).columns:
    mode_value = df_CTO_cleaned_interpolated[col].mode()[0]
    df_CTO_cleaned_interpolated[col].fillna(mode_value, inplace=True)

C:\Users\UX530\AppData\Local\Temp\ipykernel_16340\3646763603.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_CTO_cleaned_interpolated[col].fillna(mode_value, inplace=True)


In [127]:
# Comprobamos que no quedan nulos
total_rows = len(df_CTO_cleaned_interpolated)
null_summary = df_CTO_cleaned_interpolated.isna().sum().to_frame("Nulos")
null_summary["% Nulos"] = (null_summary["Nulos"] / total_rows) * 100
print(null_summary)

                        Nulos  % Nulos
ID                          0      0.0
CTO                         0      0.0
Código CTO                  0      0.0
Estado                      0      0.0
Tipo de ampliación          0      0.0
EC                          0      0.0
Fecha de solicitud          0      0.0
Fecha de ejecución          0      0.0
Fecha de documentación      0      0.0
Fecha de finalización       0      0.0
SLA                         0      0.0
Geotipo                     0      0.0
Provincia                   0      0.0
Población                   0      0.0
Zona                        0      0.0
Fase                        0      0.0
Cluster                     0      0.0
Proveedor OLT               0      0.0
Activo                      0      0.0


In [ ]:
#df_CTO_cleaned.columns
#df_CTO_cleaned['Fecha de solicitud'] = pd.to_datetime(df_CTO_cleaned['Fecha de solicitud'], errors='coerce', format='%d/%m/%Y')
#df_CTO_cleaned['Fecha de solicitud']